In [50]:
from pydataset import data
import pandas as pd
import sklearn
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Load the tips dataset.

In [5]:
#Load the tips dataset
df = data('tips')

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 1 to 244
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 15.2+ KB


In [8]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


## Create a column named price_per_person. This should be the total bill divided by the party size.

In [11]:
#Create a dataframe that holds the price per person data calculated off of the total bill divided by the siz
df['price_per_person'] = df.total_bill / df['size']
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


In [10]:
df.size

1952

## Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

probably the total bill and size columns

## Use select k best to select the top 2 features for predicting tip amount. What are they?

In [16]:
train_val, test = train_test_split(df, train_size=.8, random_state=123)

In [21]:
train, validate = train_test_split(train_val, train_size=.75, random_state=123)

In [22]:
print(f"""
train -> {train.shape}

validate -> {validate.shape}

test -> {test.shape}
""")


train -> (146, 8)

validate -> (49, 8)

test -> (49, 8)



In [43]:
X_train, y_train = train[['total_bill', 'size', 'price_per_person']], train.tip

X_validate, y_validate = validate[['total_bill', 'size', 'price_per_person']], validate.tip

X_test, y_test = test[['total_bill', 'size', 'price_per_person']], test.tip

In [44]:
#Make the thing
model = SelectKBest(f_regression, k=2)

In [45]:
#Fit the thing
model.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x125dc12d0>)

In [46]:
mask = model.get_support()

In [47]:
selected_features = X_train.columns[mask]
selected_features

Index(['total_bill', 'size'], dtype='object')

## Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [52]:
# Create a model for feature selection
model = LinearRegression()

# Create an RFE instance
rfe = RFE(estimator=model, n_features_to_select=2)

# Fit the RFE to the training data
rfe.fit(X_train, y_train)

# Get the mask of selected features
selected_mask = rfe.support_

# Get the names of the selected features
selected_features = X_train.columns[selected_mask]

print(selected_features)

Index(['total_bill', 'price_per_person'], dtype='object')


## Why do you think select k best and recursive feature elimination might give different answers for the top features? 

## Does this change as you change the number of features you are selecting?

## Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [89]:
def select_kbest(predictors, target):
    #Make the thing
    model = SelectKBest(f_regression, k=3)
    #Fit the thing
    model.fit(X_train, y_train)
    mask = model.get_support()
    selected_features = X_train.columns[mask]
    return list(selected_features)

In [55]:
select_kbest(X_train, y_train)

Index(['total_bill', 'size'], dtype='object')

## Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [87]:
def rfe(predictors, target):
    # Create a model for feature selection
    model = LinearRegression()
    rfe = RFE(estimator=model, n_features_to_select=3)
    
    #
    rfe.fit(X_train, y_train)
    selected_mask = rfe.support_
    selected_features = X_train.columns[selected_mask]
    return list(selected_features)

In [57]:
rfe(X_train, y_train)

Index(['total_bill', 'price_per_person'], dtype='object')

## Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [58]:
swiss_df = data('swiss')

In [60]:
swiss_df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [65]:
cols = swiss_df.columns.tolist()

In [74]:
def col_rename(df):
    for col in cols:
        df = df.rename(columns={col: col.replace('.','_').lower()})

In [75]:
col_rename(swiss_df)

In [76]:
swiss_df.head()

,fertility,agriculture,examination,education,catholic,infant_mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [91]:
swiss_df.examination.nunique()

22

In [86]:
data('swiss', 1)

swiss

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Swiss Fertility and Socioeconomic Indicators (1888) Data

### Description

Standardized fertility measure and socio-economic indicators for each of 47
French-speaking provinces of Switzerland at about 1888.

### Usage

    data(swiss)

### Format

A data frame with 47 observations on 6 variables, each of which is in percent,
i.e., in [0,100].

[,1] Fertility Ig, "common standardized fertility measure" [,2] Agriculture
[,3] Examination nation [,4] Education [,5] Catholic [,6] Infant.Mortality
live births who live less than 1 year.

All variables but 'Fert' give proportions of the population.

### Source

Project "16P5", pages 549-551 in

Mosteller, F. and Tukey, J. W. (1977) “Data Analysis and Regression: A Second
Course in Statistics”. Addison-Wesley, Reading Mass.

indicating their source as "Data used by permission of Franice van de Walle.
Office of Population Research, Princeton Univer

In [80]:
train_val, test = train_test_split(swiss_df, train_size=.8, random_state=123)

In [81]:
train, validate = train_test_split(train_val, train_size=.75, random_state=123)

In [93]:
print(f"""
train shape: {train.shape}
validate shape: {validate.shape}
test shape: {test.shape}
""")


train shape: (27, 6)
validate shape: (10, 6)
test shape: (10, 6)



In [83]:
X_train, y_train = train[['fertility','agriculture','catholic']], train.infant_mortality

X_validate, y_validate = validate[['fertility','agriculture','catholic']], validate.infant_mortality

X_test, y_test = test[['fertility','agriculture','catholic']], test.infant_mortality

In [90]:
select_kbest(X_train, y_train)

['fertility', 'agriculture', 'catholic']

In [88]:
rfe(X_train, y_train)

['fertility', 'agriculture', 'catholic']